In [1]:
import os
import sys
from google.colab import drive 
# Enlazar a la carpeta 'data'
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/ICBF/data')
sys.path.insert(0, '../scripts/0_utils')
!pwd

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/ICBF/data


# **Reporte de evolución nutricional**

Genera un tablero de seguimiento nutricional, incluyendo un gráfico de dispersión de los valores de peso y talla reportados para cada individuo, similar a las trabajadas por la [Organización Mundial de la Salud](https://www.who.int/tools/child-growth-standards/standards/weight-for-length-height) con anotaciones sobre cada valoración nutricional pasada.
El tablero incluye la probabilidad de que el beneficiario sufra de desnutrición aguda en el corto plazo (uno o tres meses) de acuerdo a los modelos predictivos implementados. 

####**Requerimientos:**

- Tabla de datos `app/puntos_peso_talla.csv` con los puntos para generar las curvas peso-talla para ambos sexos.
- Modelos serializados `models/model_min_i.pickle`, con `i` = 1, 3, que predice si un beneficiario que ha sufrido de desnutrición pero no está desnutrido en la actualidad recaerá después de 1 o 3 meses, en función de sus z-scores peso/talla, peso/edad e IMC actuales e inmediatamento anteriores. 
- Lista serializada `models/model_min_1_cols.pickle`, de las variables nutricionales utilizadas por ambos modelos como predictores.
- Tabla de datos nutricionales limpia `clean_data/tomas_nutricionales.parquet`, para probar el reporte.
- Función `predict_set(dataset, model, col_model, depth=2)` que retorna una tabla de relación entre el identificador `IdBeneficiario` de cada beneficiario y su probabilidad de sufrir desnutrición (*p-score*) en el futuro, de acuerdo  al conjunto de datos y al modelo ingresado.
- Función `predict_IdB(data, IdBeneficiario, model, col_model, depth=2)` que retorna la probabilidad de sufrir desnutrición de un usuario específico, de acuerdo al conjunto de datos y al modelo ingresado.

Ambas funciones requieren la librería `re` para su correcto funcionamiento.
 
####**Salidas:**

Ninguna. Define la función `nutrition_monitoring_plot(IdBeneficiario, dataset, points, lang='english')` que retorna la gráfica de evolución nutricional de un beneficiario (diagrama de dispersión peso vs talla) con los puntos coloreados según el estado nutricional peso/talla que presentó en cada valoración, a partir de las tomas presentes en un dataset. Se incluyen las curvas de z-score peso/talla enteros $z$ con $-3 \le z \le 3$ para permitir una mejor interpretación.

In [2]:
%%capture
!pip install dash
!pip install dash_bootstrap_components
!pip install jupyter-dash
#!pip install dash-renderer

In [3]:
# Librerias relevantes
import time
import dash
import plotly.express as px
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
from dash import html, dcc
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import re
import plotly.express as px
import plotly.graph_objects as go
import joblib
# Opciones de visualización
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100
pd.options.mode.chained_assignment = None

/usr/local/lib/python3.7/dist-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


### **Preliminares**

In [4]:
t0 = time.time()
# Lee la tabla de datos sociodemográficos 
points = pd.read_csv('app/puntos_peso_talla.csv')
# Lee la tabla de datos nutricionales limpia
tom = pd.read_parquet('clean_data/tomas_nutricionales.parquet')
# Carga los modelos mínimos de predicción a partir de tomas anterior y trasanterior
clf_1 = joblib.load('models/model_min_1.pickle')
clf_3 = joblib.load('models/model_min_3.pickle')
# Columnas usadas por los modelos como predictores
cols_clf = joblib.load('models/model_min_1_cols.pickle')

In [5]:
zscores = [i - 3 for i in range(7)]
color_palette = ['green', 'goldenrod', 'red', 'black']
color_estado = {tom.EstadoPesoTalla.cat.categories[i]: color_palette[abs(i - 3)] 
                for i in range(7)}
cols_estado = [col for col in tom.columns if col[:6] == 'Estado']

# Las siguientes definiciones pueden colocarse antes de ejecutar la función de graficación
z_scores = [i - 3 for i in range(7)]
color_palette = ['green', 'goldenrod', 'red', 'black']
color_estado = {
    'Desnutrición aguda severa': 'black',
    'Desnutrición aguda moderada': 'red',
    'Riesgo de desnutrición aguda': 'goldenrod',
    'Peso adecuado para la talla': 'green',
    'Riesgo de sobrepeso': 'goldenrod',
    'Sobrepeso': 'red',
    'Obesidad': 'black',
    }
cols_num_plot = ['EdadMeses', 'Talla', 'Peso', 'ZScorePesoTalla', 'EstadoPesoTalla']
cols_plot = ['Sexo', 'FechaValoracionNutricional'] + cols_num_plot
english = {
  'Desnutrición aguda severa': 'Severe wasting',
  'Desnutrición aguda moderada': 'Wasting',
  'Riesgo de desnutrición aguda': 'Risk of wasting',
  'Peso adecuado para la talla': 'Normal weight for height',
  'Riesgo de sobrepeso': 'Risk of overweight',
  'Sobrepeso': 'Overweight',
  'Obesidad': 'Obesity',
  'plot title': 'Nutritional monitoring',
  'x axis': 'Height (cm)',
  'y axis': 'Weight (kg)',
  'legend title': 'Nutritional status (Weight-for-height)'
  }
spanish = {
  'Desnutrición aguda severa': 'Desnutrición aguda severa',
  'Desnutrición aguda moderada': 'Desnutrición aguda moderada',
  'Riesgo de desnutrición aguda': 'Riesgo de desnutrición aguda',
  'Peso adecuado para la talla': 'Peso adecuado para la talla',
  'Riesgo de sobrepeso': 'Riesgo de sobrepeso',
  'Sobrepeso': 'Sobrepeso',
  'Obesidad': 'Obesidad',
  'plot title': 'Seguimiento nutricional',
  'x axis': 'Talla (cm)',
  'y axis': 'Peso (kg)',
  'legend title': 'Estado nutricional (Peso/Talla)'
  }
# Variables para el reporte (no son necesarias para el gráfico)
cols_reporte = [
  'FechaNacimiento',
  'Sexo',
  'FechaValoracionNutricional',
  'EdadMeses',
  'Talla',
  'Peso',
  'EstadoPesoTalla',
  'EstadoPesoEdad',
  'EstadoIMC',
  'EstadoTallaEdad']

### **Función de predicción**

In [6]:
# Función de predicción para un conjunto de datos
def predict_set(dataset, model, col_model, depth=2):
  ''' 
  Calcula la probabilidad de recurrencia en desnutrición de los beneficiarios de
  un tabla de datos 'dataset' que tengan 'depth' o más tomas nutricionales, de acuerdo
  al modelo predictivo 'model', utilizando las columnas 'col_model' como predictores.
  Retorna una tabla de relación entre cada identificador 'IdBeneficiario' y su respectivo
  'p_score' (probabilidad de sufrir desnutrición).
  import pandas as pd
  import numpy as np
  import re
  '''
  dias_mes = 365.25 / 12
  df = dataset.sort_values(['IdBeneficiario', 'EdadMeses'])
  cols_imp = ['IdBeneficiario'] + list({re.sub('-\d$', '', col) for col in col_model})
  df = df.dropna(subset=cols_imp).reset_index(drop=True)
  if len(df) > 0:
    his = df[cols_imp]
    # Para cada periodo tomado en cuenta (de 1 a 'depth')
    for i in range(depth):
      periodo = i + 1
      # Coloca al frente de cada fila, los datos de la fila que está 'periodo' filas antes
      for col in cols_imp:
        his[f'{col}-{periodo}'] = his[col].shift(periodo)
    # Conserva solo las filas donde todos los datos pertenecen al mismo beneficiario
    his = his.loc[his['IdBeneficiario'] == his[f'IdBeneficiario-{depth}']]
    # Calcula el tiempo desde la última toma redondeado al número de meses
    his['tiempo'] = (his['EdadMeses'] - his['EdadMeses-1']).round(0)
    # Deja solo las columnas de la predicción y el identificador del beneficiario
    his = his[['IdBeneficiario'] + col_model].\
      drop_duplicates(subset=['IdBeneficiario'], keep='last')
    IdBeneficiario = his['IdBeneficiario']
    X = his.drop(columns=['IdBeneficiario'])
    y = [q for p, q in model.predict_proba(X)]
    prediction = pd.DataFrame({'IdBeneficiario': IdBeneficiario, 'p_score': y})
    prediction = prediction.drop_duplicates(subset=['IdBeneficiario'], keep='last')
    return prediction
  else:
    print('Datos insuficientes.')
    return np.nan
# Función de predicción para un individuo
def predict_IdB(data, IdBeneficiario, model, col_model, depth=2):
  ''' 
  Calcula la probabilidad de recurrencia en desnutrición de un beneficiario, a partir
  de su identificador 'IdBeneficiario' y sus registros en un tabla de datos 'dataset' 
  que tengan 'depth' o más tomas nutricionales del beneficiario, de acuerdo al modelo
  predictivo 'model', utilizando las columnas 'col_model' como predictores.
  Retorna el respectivo 'p_score' (probabilidad de sufrir desnutrición) asociado al
  beneficiario.
  De no ser posible realizar la predicción, retorna un NaN.
  import pandas as pd
  import numpy as np
  import re
  import predict_set
  '''
  # Registros del beneficiario
  subset = data[data['IdBeneficiario'] == IdBeneficiario]
  try:
    # Función de predicción
    return predict_set(subset, model, col_model, depth=depth)['p_score'].iloc[0]
  except:
    # Si no es posible hallar el p-score, retorna un NaN
    print('Datos insuficientes.')
    return np.nan

### **Función de graficación**

In [7]:
def nutrition_monitoring_plot(IdBeneficiario, dataset, points, lang='english'):
  '''Realiza la gráfica de seguimiento nutricional para un beneficiario, a partir de su 
  identificador IdBeneficiario en un dataset de tomas nutricionales, incluyendo:
  'Sexo', 'Talla', 'Peso', 'EstadoPesoTalla', 'EstadoPesoTalla', 'ZScorePesoTalla', 'EdadMeses',
  'FechaValoracionNutricional', en el idioma seleccionado (inglés o español)
  import pandas as pd
  import numpy as np
  import plotly.express as px
  import plotly.graph_objects as go
  '''
  # Selecciona los datos del beneficiario para la gráfica
  df = dataset.loc[dataset['IdBeneficiario'] == IdBeneficiario, cols_plot].\
    dropna().sort_values('FechaValoracionNutricional')
  df[cols_num_plot] = df[cols_num_plot].round(2)
  # Define el sexo y el lenguage
  sex = df['Sexo'].iloc[0]
  if lang == 'spanish':
    language = spanish
  else:
    language = english
  # Diagrama de dispersión peso vs talla
  fig = px.scatter(df, x='Talla', y='Peso', color='EstadoPesoTalla',
                  hover_data=['EstadoPesoTalla', 'ZScorePesoTalla', 'EdadMeses'],
                  hover_name='FechaValoracionNutricional',
                  color_discrete_map=color_estado)
  # Parámetros del gráfico
  fig.update_traces(marker=dict(size=10))
  fig.layout = dict(paper_bgcolor='white',
                    plot_bgcolor='white',
                    xaxis = dict(title = language['x axis'],
                                 showgrid=True,
                                 gridwidth=1,
                                 gridcolor='gray',
                                 range=[45, 120],
                                 mirror=True,
                                 ticks='outside',
                                 showline=True,
                                 linewidth=2,
                                 linecolor='black',
                                 tickvals=[5 * i for i in range(9, 25)]),
                    yaxis = dict(title = language['y axis'],
                                 showgrid=True,
                                 gridwidth=1,
                                 gridcolor='gray',
                                 range=[0, 34],
                                 mirror=True,
                                 ticks='outside',
                                 showline=True,
                                 linewidth=2,
                                 linecolor='black',
                                 tickvals=[2 * i for i in range(18)]),
                    font=dict(size=14),
                    legend=dict(bgcolor='white',
                                borderwidth=1))
  # Título del gráfico
  fig.update_layout(title= f"{language['plot title']}: {IdBeneficiario}")
  # Actualiza el nombre de las categorías según el idioma seleccionado
  fig.for_each_trace(lambda t: t.update(name = language[t.name]))
  # Agrega los puntos (talla, peso) con cada punto coloreado de acuerdo al estado peso/talla
  fig.add_trace(go.Scatter(x=df['Talla'], y=df['Peso'],
                           mode='lines', showlegend=False,
                           line=dict(color='black', width=1)))
  # Traza las curvas de z-score entero para -3 <= z <= 3
  for z in z_scores:
    fig.add_trace(go.Scatter(x=points['height'], y=points[f'weight_{sex}_{z}'],
                             hoverinfo = 'none', showlegend=False,
                             line=dict(color=color_palette[abs(z)], width=1)))
  return fig

In [8]:
# Prueba de la función de graficación
nutrition_monitoring_plot(IdBeneficiario=68400, 
                          dataset=tom,
                          points=points)

### **Aplicación de reporte en `JupyterDash`**

In [9]:
# Inicializa la aplicación Dash en Jupyter
app = JupyterDash(__name__)

def nutritional_monitoring_report(IdBeneficiario, dataset, points, lang_plot='english'):
  '''
  Inicializa una aplicación en JupyterDash y genera un reporte nutricional para el beneficiario incluyendo:
  - Datos generales del beneficiario: IdBeneficiario, fecha de nacimiento, sexo y edad
  - Datos de la última toma: fecha, edad, talla, peso y estados nutricionales
  - Gráfico de seguimiento nutricional
  - Probabilidad de caer en desnutrición en los próximos 1 y 3 meses
  '''
  df = tom[tom['IdBeneficiario'] == IdBeneficiario]
  # Datos del último reporte del beneficiario
  nacimiento, sexo, fecha_valoracion, edad, talla, peso, peso_talla, peso_edad, IMC, talla_edad = \
    tuple(df[cols_reporte].iloc[-1])
  # Cálculo de probabilidad de caer en desnutrición aguda según los modelos predictivos
  p_score_1 = predict_IdB(dataset, IdBeneficiario, clf_1, cols_clf)
  p_score_3 = predict_IdB(dataset, IdBeneficiario, clf_3, cols_clf)

  fig = nutrition_monitoring_plot(IdBeneficiario=IdBeneficiario,
                                  dataset=dataset,
                                  points=points,
                                  lang=lang_plot)
  app.layout = html.Div(children=[
      # Datos del beneficiario
      html.H1(children='Evolución estado nutricional'), 
      html.Div(children=f"Identificador: {IdBeneficiario}"),
      html.Div(children=f"Fecha de Nacimiento: {nacimiento:%Y-%m-%d}"), 
      html.Div(children=f"Sexo: {sexo}"), 
      html.Div(children=f"Edad: {edad:.1f} meses"),
      # Datos de la última revisión 
      html.H3(children='Última revisión'),
      html.Div(children=f"Fecha: {fecha_valoracion:%Y-%m-%d}"),
      html.Div(children=f"Edad: {edad:.1f} meses"),
      html.Div(children=f"Talla: {talla:.1f} cm"),
      html.Div(children=f"Peso: {peso:.1f} kg"),
      html.Div(children=f"Estado Peso/Talla: {peso_talla}"),
      html.Div(children=f"Estado Peso/Edad: {peso_edad}"),
      html.Div(children=f"Estado IMC: {IMC}"),
      html.Div(children=f"Estado Talla/Edad: {talla_edad}"),
      # Figura de seguimiento
      dcc.Graph(
          id='example-graph',
          figure=fig
      ),
      # Predicción del riesgo de acuerdo a los modelos
      html.H3(children='Riesgo de caer en desnutrición aguda'),
      html.Div(children=f"En el próximo mes: {p_score_1:.0%}"),
      html.Div(children=f"En los próximos tres meses: {p_score_3:.0%}"),
  ])
  app.run_server(mode='inline')

In [10]:
# Prueba de la función de reporte
nutritional_monitoring_report(12600290,
                             tom,
                             points,
                             lang_plot='spanish')

<IPython.core.display.Javascript object>